IMPORTING LIBRARIES

In [ ]:
import requests
import re

from bs4 import BeautifulSoup

from langchain.document_loaders    import UnstructuredFileLoader
from langchain.text_splitter       import CharacterTextSplitter
from langchain.vectorstores.chroma import Chroma

from langchain.embeddings.huggingface    import HuggingFaceEmbeddings
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

from transformers.pipelines import pipeline

DATA EXTRACTION FROM SPECIFIED URLS

In [2]:
urls = ['https://fctraining.org/complete-seo-course.php', 
        'https://fctraining.org/data-science-machine-learning-ai-course.php']

data = ''
for url in urls:

  rp = requests.get(url)
  bs = BeautifulSoup(rp.text, 'html.parser')

  dt = re.sub(r'\[.*?\]', '', bs.get_text())
  dt = dt.strip()

  data += (dt + '\n\n')

with open('Chat_Bot_Data.txt', 'w', encoding = 'utf-8') as file:
  file.write(data)

print('Data Extracted Successfully.\n')

Data Extracted Successfully.



FINE TUNING A HUGGING FACE MODEL

In [6]:
ld = UnstructuredFileLoader('Chat_Bot_Data.txt')
dm = ld.load() 

ts = CharacterTextSplitter(chunk_size = 100, chunk_overlap = 0, separator = '\n\n', length_function = len)
ds = ts.split_documents(dm)

eb = HuggingFaceEmbeddings()
vs = Chroma.from_documents(ds, eb)

pl = pipeline(model = 'declare-lab/flan-alpaca-base', task = 'text2text-generation', max_length = 1100, do_sample = True, temperature = 0.9)
lm = HuggingFacePipeline(pipeline = pl)

mm = ConversationBufferMemory(memory_key = 'chat_history', return_messages = True)
ch = ConversationalRetrievalChain.from_llm(llm = lm, retriever = vs.as_retriever(), memory = mm)

Created a chunk of size 255, which is longer than the specified 100
Created a chunk of size 275, which is longer than the specified 100
Created a chunk of size 207, which is longer than the specified 100
Created a chunk of size 432, which is longer than the specified 100
Created a chunk of size 180, which is longer than the specified 100
Created a chunk of size 134, which is longer than the specified 100
Created a chunk of size 224, which is longer than the specified 100
Created a chunk of size 267, which is longer than the specified 100
Created a chunk of size 210, which is longer than the specified 100
Created a chunk of size 268, which is longer than the specified 100
Created a chunk of size 291, which is longer than the specified 100
Created a chunk of size 263, which is longer than the specified 100
Created a chunk of size 265, which is longer than the specified 100
Created a chunk of size 240, which is longer than the specified 100
Created a chunk of size 272, which is longer tha

HAVING A CONVERSATION WITH THE TRAINED MODEL

In [7]:
cd = True

while cd == True:
  q = input('Q : ')

  if q == 'Exit':
    cd = False

  else:
    a = ch.run(q)
    print(f'A : {a}\n')

A : The cost of the course would depend on the size of the course, the location, the requirements for attendance, and the level of completion of the course.

A : The cost of your Data Science course will depend on the number of sessions and the course itself.

